## Load necessary modules

In [10]:
import matplotlib 
matplotlib.use('Agg')
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet.models.resnet import custom_objects
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import sys

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/scratch/user/puneetkohli/.conda/envs/puneet_keras_env/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Run RetinaNet model

In [6]:
def run_model(image, model):
    # copy to draw on
    draw = image.copy()
    print ("Original Frame is = ", draw.shape)
    
    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)
    print ("Processed Frame is = ", image.shape)
    
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)
    sys.stdout.flush()
    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break
        print ("Found label = ", label)
        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)

    fig = plt.figure(figsize=(12.8, 7.2))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    plt.imshow(draw)
    fig.canvas.draw()
    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='') #canvas to string
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,)) #string to array
    data = data[:, :, ::-1]
    plt.close() 
    print("Data shape = " , data.shape)
    return data, label

## Get COCO Class Names

In [12]:
# load label to names mapping for visualization purposes
labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

## Run detection on video

In [7]:
def run_on_video(video_path, video_name, model):
    cap = cv2.VideoCapture(video_path)
    import time
    start_time = time.time()
    fps = cap.get(cv2.CAP_PROP_FPS)
    out_path = "out_" + video_name + "_RetinaNet_2" + ".avi"
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (1280,720)) #width, height format
    i = 0
    people_frames = []
    while(cap.isOpened()):
        has_frame, frame = cap.read()
        if not has_frame: 
            break
        # Custom code here
        i += 1
        print ("Processing frame #", i)
        sys.stdout.flush()
        processed_img, label = run_model(frame, model)
        out.write(processed_img)
        print ("Label = ", label)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Finished reading video")
    print ("Wrote video to ", out_path)
    print ("Frames with person", people_frames)
    sys.stdout.flush()
    cap.release()
    out.release()

## Run detection on image

In [8]:
def run_on_image(image_path, model):
    image = read_image_bgr(image_path)
    run_model(image, model)

## Perform Tests

In [13]:
# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())
## Load RetinaNet model
# adjust this to point to your downloaded/trained model
model_path = os.path.join('..', 'snapshots', 'resnet50_coco_best_v2.0.3.h5')
# load retinanet model
model = keras.models.load_model(model_path, custom_objects=custom_objects)
print("Model Loaded")
sys.stdout.flush()
#run_on_image('000000008021.jpg', model)
run_on_video("uber_trimmed.mp4", "uber_trimmed", model)

Model Loaded
Processing frame # 1


/home/puneetkohli/.local/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  1.256347417831421
Data shape =  (518, 921, 3)
Label =  2
Processing frame # 2
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.495774507522583
Data shape =  (518, 921, 3)
Label =  0
Processing frame # 3
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4960165023803711
Data shape =  (518, 921, 3)
Label =  0
Processing frame # 4
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4944002628326416
Data shape =  (518, 921, 3)
Label =  0
Processing frame # 5
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.49475789070129395
Found label =  0
Data shape =  (518, 921, 3)
Label =  2
Processing frame # 6
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.49665117263793945
Data shape =  (518, 921

Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.49522876739501953
Data shape =  (518, 921, 3)
Label =  43
Processing frame # 50
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4954690933227539
Data shape =  (518, 921, 3)
Label =  2
Processing frame # 51
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4963250160217285
Data shape =  (518, 921, 3)
Label =  66
Processing frame # 52
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.49602484703063965
Data shape =  (518, 921, 3)
Label =  66
Processing frame # 53
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.49602365493774414
Data shape =  (518, 921, 3)
Label =  66
Processing frame # 54
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4990983009338379
Data shape =  (518, 921, 3)


Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4972372055053711
Data shape =  (518, 921, 3)
Label =  2
Processing frame # 97
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.497344970703125
Data shape =  (518, 921, 3)
Label =  2
Processing frame # 98
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4968833923339844
Data shape =  (518, 921, 3)
Label =  2
Processing frame # 99
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4957127571105957
Data shape =  (518, 921, 3)
Label =  2
Processing frame # 100
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4968247413635254
Data shape =  (518, 921, 3)
Label =  2
Processing frame # 101
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.5012168884277344
Data shape =  (518, 921, 3)
Label 

Data shape =  (518, 921, 3)
Label =  66
Processing frame # 144
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4967622756958008
Data shape =  (518, 921, 3)
Label =  66
Processing frame # 145
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4975314140319824
Data shape =  (518, 921, 3)
Label =  66
Processing frame # 146
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4969055652618408
Data shape =  (518, 921, 3)
Label =  66
Processing frame # 147
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.4977905750274658
Data shape =  (518, 921, 3)
Label =  66
Processing frame # 148
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3)
processing time:  0.49631261825561523
Data shape =  (518, 921, 3)
Label =  66
Processing frame # 149
Original Frame is =  (720, 1280, 3)
Processed Frame is =  (750, 1333, 3